# Age Regression CNN — UTKFace (Aligned & Cropped)
**But** : prédire l'âge (valeur continue) à partir d'une image de visage.

- Dataset : UTKFace *Aligned & Cropped* (≈107 MB)
- Modèle : MobileNetV2 (ImageNet) + tête régression
- Perte : MSE ; Métrique : MAE (erreur moyenne en années)

In [ ]:
!nvidia-smi || echo "No NVIDIA GPU (CPU only)"
!pip -q install -U tensorflow gradio

import os, re, zipfile, random, math, json, pathlib
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
print("TF:", tf.__version__)


In [ ]:
import kagglehub, os, shutil

path = kagglehub.dataset_download("jangedoo/utkface-new")
print("Path to dataset files:", path)

DATA_ROOT = "/content/data"
EXTRACT_DIR = os.path.join(DATA_ROOT, "UTKFace_kaggle")
if os.path.exists(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)
os.makedirs(EXTRACT_DIR, exist_ok=True)


def is_img(fname):
    return fname.lower().endswith((".jpg",".jpeg",".png"))

count = 0
for root, _, files in os.walk(path):
    for f in files:
        if is_img(f):
            src = os.path.join(root, f)
            dst = os.path.join(EXTRACT_DIR, f)
            if os.path.exists(dst):
                stem, ext = os.path.splitext(f)
                i = 1
                while os.path.exists(dst):
                    dst = os.path.join(EXTRACT_DIR, f"{stem}_{i}{ext}")
                    i += 1
            shutil.copy2(src, dst)
            count += 1

print("Images copiées:", count)
print("EXTRACT_DIR =", EXTRACT_DIR)


In [ ]:
def list_images_with_age(root):
    paths, ages = [], []
    for dirpath, _, files in os.walk(root):
        for f in files:
            if f.lower().endswith((".jpg",".jpeg",".png")):
                m = re.match(r"^(\d+)_", f)
                if not m:
                    m = re.match(r"^(\d+)", f) 
                if m:
                    age = int(m.group(1))
                    if 0 <= age <= 100: 
                        paths.append(os.path.join(dirpath, f))
                        ages.append(age)
    return pd.DataFrame({"path": paths, "age": ages})

df = list_images_with_age(EXTRACT_DIR)
print("Images valides:", len(df))
df.head()


In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df, test_df = train_test_split(df, test_size=0.10, random_state=42)
train_df, val_df  = train_test_split(train_df, test_size=0.10, random_state=42)
len(train_df), len(val_df), len(test_df)


In [ ]:
IMG_SIZE = 160
BATCH = 64
AUTOTUNE = tf.data.AUTOTUNE

def decode_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE), antialias=True)
    return img

def preprocess(img):
    return tf.keras.applications.mobilenet_v2.preprocess_input(img)

def load_item(path, age):
    img = decode_image(path)
    img = preprocess(img)
    age = tf.cast(age, tf.float32)
    return img, age

augment = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augment")

def df_to_ds(frame, training=False):
    ds = tf.data.Dataset.from_tensor_slices((frame["path"].values, frame["age"].values))
    ds = ds.map(load_item, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.map(lambda x, y: (augment(x, training=True), y), num_parallel_calls=AUTOTUNE)
        ds = ds.shuffle(2048, reshuffle_each_iteration=True)
    ds = ds.batch(BATCH).prefetch(AUTOTUNE)
    return ds

train_ds = df_to_ds(train_df, training=True)
val_ds   = df_to_ds(val_df, training=False)
test_ds  = df_to_ds(test_df, training=False)

train_ds, val_ds, test_ds


In [ ]:
base = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)
base.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="linear")(x)  # âge (continu)

model = keras.Model(inputs, outputs)
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="mse",
    metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
)


In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_mae", patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=2, min_lr=1e-5)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks,
    verbose=1
)


In [ ]:
base.trainable = True
for i, layer in enumerate(base.layers):
    if i < int(0.75 * len(base.layers)):  
        layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(5e-4),
    loss="mse",
    metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
)

history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)


In [ ]:
test_loss, test_mae = model.evaluate(test_ds, verbose=0)
print(f"Test — MSE: {test_loss:.3f} | MAE: {test_mae:.3f} ans")

y_true, y_pred = [], []
for xb, yb in test_ds:
    preds = model.predict(xb, verbose=0)
    y_true.extend(yb.numpy().tolist())
    y_pred.extend(preds.squeeze().tolist())

y_true = np.array(y_true, dtype=float)
y_pred = np.clip(np.array(y_pred, dtype=float), 0, 100)

mae = np.mean(np.abs(y_true - y_pred))
within5 = np.mean(np.abs(y_true - y_pred) <= 5.0)
print(f"MAE global: {mae:.2f} ans")
print(f"Précision ±5 ans: {within5*100:.1f}%")


In [ ]:
SAVE_DIR = "/content/age_regressor"
os.makedirs(SAVE_DIR, exist_ok=True)
MODEL_PATH = os.path.join(SAVE_DIR, "age_regressor.h5")
model.save(MODEL_PATH)
print("Sauvegardé:", MODEL_PATH)

def predict_age(img_path: str) -> float:
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE), antialias=True)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    pred = model.predict(tf.expand_dims(img, 0), verbose=0)[0][0]
    return float(np.clip(pred, 0, 100))

sample_path = test_df.iloc[0]["path"]
print("Exemple:", sample_path)
print("Âge vrai:", int(test_df.iloc[0]["age"]), "| Âge estimé:", round(predict_age(sample_path), 1))


In [ ]:
import gradio as gr

def infer_gradio(img):
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE), antialias=True)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    pred = model.predict(tf.expand_dims(img, 0), verbose=0)[0][0]
    age = float(np.clip(pred, 0, 100))
    return f"Estimated age: {age:.1f} years"

demo = gr.Interface(
    fn=infer_gradio,
    inputs=gr.Image(type="pil", label="Upload a cropped face image"),
    outputs=gr.Textbox(label="Prediction"),
    title="Age Regression (UTKFace via kagglehub)",
    description="Upload a cropped face image. The model returns an estimated age (continuous)."
)
demo.launch(debug=False, share=True)
